In [1]:
import os
import re # Regular expression matching
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import sys, os, glob, pickle, gc
import math

mpl.rcParams['figure.figsize'] = (12,9)
mpl.rcParams['font.size'] = 26


In [2]:
## ## ## Setup colormap ## ## ##
# define the colormap
cmap = plt.cm.jet
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5,.5,.5,1.0)
# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.linspace(-7.5,0,16)
ticks  =[-7,-6,-5,-4,-3,-2,-1,0]
norm   = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [40]:
#locFiles  = ['spLoc_16.00.txt','spLoc_12.00.txt','spLoc_10.00.txt','spLoc_08.00.txt','spLoc_05.00.txt' ]
#massFiles = ['spMass_16.00.txt','spMass_12.00.txt','spMass_10.00.txt','spMass_08.00.txt','spMass_05.00.txt']
#ZFiles    = ['spZ_16.00.txt','spZ_12.00.txt','spZ_10.00.txt','spZ_08.00.txt','spZ_05.00.txt']
#ppfFiles  = ['spPPF_16.00.txt','spPPF_12.00.txt','spPPF_10.00.txt','spPPF_08.00.txt','spPPF_05.00.txt']
#ppzFiles  = ['spPZ_16.00.txt','spPZ_12.00.txt','spPZ_10.00.txt','spPZ_08.00.txt','spPZ_05.00.txt']
locFiles  = locF = 'spLoc_05.00.txt'
massFiles = massF = 'spMass_05.00.txt'
ZFiles    = ZF = 'spZ_05.00.txt'
ppfFiles  = ppfF = 'spPPF_05.00.txt'
ppzFiles  = pzfF = 'spPZ_05.00.txt'

In [41]:
zs=[5.00000000000000000e+00, 7.029400000000000546e+02]
z=5;bs=702.94

In [42]:
dotNorm = 10.0  # For dot-size scaling
comovbox = 5.0 
i = 0

In [43]:
sbox = comovbox / (1.0 + z) * 0.71 # Create a box that's sbox kpc physical
print ("z=%.3lf"%z)
spPosExp = r'z%05.2lf_SpCoord_[0-9]*.txt' %z # These are the x,y,z locations of interest from the genStarPlots6.py
poisFiles  = [f for f in os.listdir('.') if re.match(spPosExp, f)]
poisFiles.sort()

z=5.000


In [44]:
poisFiles

['z05.00_SpCoord_0.txt',
 'z05.00_SpCoord_1281414.txt',
 'z05.00_SpCoord_2562828.txt',
 'z05.00_SpCoord_3844242.txt']

In [82]:
poi=poisFiles[2]
index=2
poi, locF

('z05.00_SpCoord_2562828.txt', 'spLoc_05.00.txt')

In [46]:
locs = np.loadtxt(locF,skiprows=1) 
mass = np.loadtxt(massF,skiprows=1)
Z    = np.loadtxt(ZF,skiprows=1)

In [90]:
ppf  = np.loadtxt(ppfF,skiprows=1)
pzf  = np.loadtxt(pzfF,skiprows=1)

In [91]:
Z[Z<1.0e-5] = 1.0e-10
pzf[pzf<1.0e-5] = 1.0e-10

In [92]:
x,y,zz = np.loadtxt(poi)  # star particle location of interest
xo,yo,zo = x,y,zz
print( "Star offset [%.2lf %.2lf %.2lf]"%(x,y,zz))
print( "Boxsize at this z (physical) %.4lf"%bs)
print( "Our 5 kpc (comov) box at this z (physical) %.4lf"%sbox)

Star offset [133.13 170.87 275.48]
Boxsize at this z (physical) 702.9400
Our 5 kpc (comov) box at this z (physical) 0.5917


In [93]:
        if (abs(x) > bs/2.0 - sbox/2.0):
            x = np.sign(x) * (bs/2.0 - sbox/2.0)
        if (abs(y) > bs/2.0 - sbox/2.0):
            y = np.sign(y) * (bs/2.0 - sbox/2.0)
        if (abs(zz) > bs/2.0 - sbox/2.0):
            zz = np.sign(zz) * (bs/2.0 - sbox/2.0)
        print( "Star offset adjusted for boxsize [%.2lf %.2lf %.2lf]"%(x,y,zz))

Star offset adjusted for boxsize [133.13 170.87 275.48]


In [94]:
xmin = x - sbox/2.0
xmax = x + sbox/2.0
ymin = y - sbox/2.0
ymax = y + sbox/2.0
zmin = zz - sbox/2.0 
zmax = zz + sbox/2.0
print( "Box range x(%.2lf,%.2lf), y(%.2lf,%.2lf), z(%.2lf,%.2lf)"%(xmin,xmax,ymin,ymax,zmin,zmax))

Box range x(132.83,133.43), y(170.57,171.17), z(275.18,275.78)


In [95]:
xcond = ((locs[:,0] >= xmin) & (locs[:,0] <= xmax))
ycond = ((locs[:,1] >= ymin) & (locs[:,1] <= ymax))
zcond = ((locs[:,2] >= zmin) & (locs[:,2] <= zmax))

In [96]:
locsFilt = locs[xcond & ycond & zcond] # filter: just get stars in plot region
print(len(locsFilt))

14651


In [97]:
massFilt = mass[xcond & ycond & zcond] # filter: just get stars in plot region
Zfilt    = Z[xcond & ycond & zcond] # filter: just get stars in plot region
ppfFilt  = ppf[xcond & ycond & zcond] # filter: just get stars in plot region
pzfFilt  = pzf[xcond & ycond & zcond] # filter: just get stars in plot region

In [98]:
locsFilt = locsFilt - np.array([x,y,zz])

In [99]:
rng1 = (Zfilt < 1.e-5)
rng2 = ((Zfilt >= 1.e-5) & (Zfilt < 1.e-3))
rng3 = ((Zfilt >= 1.e-3) & (Zfilt < 1.e-1))
rng4 = (Zfilt >= 1.e-1)
print(Zfilt[rng4])

[ 0.11007154  0.11111382]


In [61]:
massFilt[Zfilt>.1]
np.sum(massFilt[Zfilt>.1])

1451.1827163022408

In [62]:
z1=np.log10(Zfilt[rng1])
z2=np.log10(Zfilt[rng2])
z3=np.log10(Zfilt[rng3])
z4=np.log10(Zfilt[rng4])

In [63]:
# Create the starlocation Z plot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')
# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in [ax1,ax2]], visible=False)
plt.setp([a.get_yticklabels() for a in [ax2,ax4]], visible=False)
plt.setp([a.set_xlim([-sbox/2.0,sbox/2.0]) for a in [ax1,ax2,ax3,ax4]])
plt.setp([a.set_ylim([-sbox/2.0,sbox/2.0]) for a in [ax2,ax4,ax3,ax4]])

In [64]:
xcoord = locsFilt[:,0]; ycoord = locsFilt[:,1]
ax1.scatter(xcoord[rng1], ycoord[rng1], s=massFilt[rng1]/dotNorm, c=z1, cmap=cmap,vmin=-7.5, vmax=0)
ax2.scatter(xcoord[rng2], ycoord[rng2], s=massFilt[rng2]/dotNorm, c=z2, cmap=cmap,vmin=-7.5, vmax=0)
ax3.scatter(xcoord[rng3], ycoord[rng3], s=massFilt[rng3]/dotNorm, c=z3, cmap=cmap,vmin=-7.5, vmax=0)
ax4.scatter(xcoord[rng4], ycoord[rng4], s=massFilt[rng4]/dotNorm, c=z4, cmap=cmap,vmin=-7.5, vmax=0)

In [65]:
ax5 = fig.add_axes([0.85, 0.1, 0.025, 0.85])
cb = mpl.colorbar.ColorbarBase(ax5, cmap=cmap, norm=norm, spacing='proportional',
    ticks=ticks, boundaries=bounds, format='%1i')

In [66]:
        xpos = ax1.get_xlim()[0] - 0.07 * ax1.get_xlim()[0]
        ypos = ax1.get_ylim()[0] - 0.08 * ax1.get_ylim()[0]
        bbox = {'facecolor':'white', 'alpha':0.75, 'pad':3}
        ax1.text(xpos,ypos,'$Z_{\odot} <\, 10^{-5}$',
                 bbox=bbox, fontsize=20)
        ax2.text(xpos,ypos,'$10^{-5} \leq\, Z_{\odot}\, <\, 10^{-3}$',
                 bbox=bbox,fontsize=20)
        ax3.text(xpos,ypos,'$10^{-3} \leq\, Z_{\odot}\, <\, 10^{-1}$',
                 bbox=bbox,fontsize=20)
        ax4.text(xpos,ypos,'$10^{-1} \leq\, Z_{\odot}$',
                 bbox=bbox,fontsize=20)

In [67]:
        startx, endx = ax1.get_xlim(); dx = (endx-startx) * 0.1
        starty, endy = ax1.get_ylim(); dy = (endy-starty) * 0.1
        formatter = FormatStrFormatter('%.2f')
        ax1.yaxis.set_ticks([starty+dy, 0,endy-dy]); ax1.yaxis.set_major_formatter(formatter)
        ax3.yaxis.set_ticks([starty+dy, 0,endy-dy]); ax3.yaxis.set_major_formatter(formatter)
        ax3.xaxis.set_ticks([startx+dx, 0,endx-dx]); ax3.xaxis.set_major_formatter(formatter)
        ax4.xaxis.set_ticks([startx+dx, 0,endx-dx]); ax4.xaxis.set_major_formatter(formatter)
        ax3.set_xlabel('x kpc')
        ax4.set_xlabel('x kpc')
        ax1.set_ylabel('y kpc')
        ax3.set_ylabel('y kpc')
        ax5.set_ylabel('$log\; Z_{\odot}$', size=34)

In [68]:
plt.subplots_adjust(left=0.15, bottom=0.1, right=0.84, top=.95, wspace=.05, hspace=.05)
plt.show()

In [102]:
Z[(Z>.1) & xcond & ycond & zcond],mass[(Z>.1) & xcond & ycond & zcond]

(array([ 0.11007154,  0.11111382]), array([ 725.59135815,  725.59135815]))

In [74]:
print( "Box range x(%.2lf,%.2lf), y(%.2lf,%.2lf), z(%.2lf,%.2lf)"%(xmin,xmax,ymin,ymax,zmin,zmax))

Box range x(132.83,133.43), y(170.57,171.17), z(275.18,275.78)


In [100]:
np.savetxt("4PanelPlotMassTotals_z=%.1lf-%i.txt"%(z,indx),massFilt,comments='')
np.savetxt("4PanelPlotPopIIIMassTotals_z=%.1lf-%i.txt"%(z,indx),massFilt * ppfFilt,comments='')
np.savetxt("4PanelPlotPZMassTotals_z=%.1lf-%i.txt"%(z,indx), (1.0-ppfFilt * massFilt) * 
                                   (ppfFilt / Zfilt) * massFilt,comments='')

In [101]:
massFilt * ppfFilt

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])